In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import plotly.express as px

In [3]:
# 1
df = pd.read_csv("gapminder_clean.csv")
df

Unnamed: 0 Country Name  Year  Agriculture, value added (% of GDP)  \
0              0  Afghanistan  1962                                  NaN   
1              1  Afghanistan  1967                                  NaN   
2              2  Afghanistan  1972                                  NaN   
3              3  Afghanistan  1977                                  NaN   
4              4  Afghanistan  1982                                  NaN   
...          ...          ...   ...                                  ...   
2602        2602     Zimbabwe  1987                            14.407528   
2603        2603     Zimbabwe  1992                             7.413793   
2604        2604     Zimbabwe  1997                            18.934082   
2605        2605     Zimbabwe  2002                            14.029007   
2606        2606     Zimbabwe  2007                            21.597907   

      CO2 emissions (metric tons per capita)  \
0                                   0.073781   
1                                   0.123782   
2                                   0.130820   
3                                   0.183118   
4                                   0.165879   
...                                      ...   
2602                                1.598217   
2603                                1.533724   
2604                                1.194678   
2605                                0.942795   
2606                                0.743449   

      Domestic credit provided by financial sector (% of GDP)  \
0                                             21.276422         
1                                              9.917662         
2                                             18.880833         
3                                             13.836822         
4                                                   NaN         
...                                                 ...         
2602                                          74.161607         
2603                                          43.120518         
2604                                          63.058320         
2605                                         164.559047         
2606                                                NaN         

      Electric power consumption (kWh per capita)  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   
...                                           ...   
2602                                   878.072691   
2603                                   778.695133   
2604                                   870.988697   
2605                                   827.329873   
2606                                   700.642317   

      Energy use (kg of oil equivalent per capita)  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   
...                                            ...   
2602                                    896.673611   
2603                                    923.493407   
2604                                    804.508892   
2605                                    772.676619   
2606                                    725.681049   

      Exports of goods and services (% of GDP)  \
0                                     4.878051   
1                                     6.772908   
2                                    14.763231   
3                                    11.662904   
4                                          NaN   
...                                        ...   
2602                                 24.015710   
2603                                 27.227263   

In [4]:
df.dtypes

Unnamed: 0                                                   int64
Country Name                                                object
Year                                                         int64
Agriculture, value added (% of GDP)                        float64
CO2 emissions (metric tons per capita)                     float64
Domestic credit provided by financial sector (% of GDP)    float64
Electric power consumption (kWh per capita)                float64
Energy use (kg of oil equivalent per capita)               float64
Exports of goods and services (% of GDP)                   float64
Fertility rate, total (births per woman)                   float64
GDP growth (annual %)                                      float64
Imports of goods and services (% of GDP)                   float64
Industry, value added (% of GDP)                           float64
Inflation, GDP deflator (annual %)                         float64
Life expectancy at birth, total (years)                    flo

In [5]:
# 2
filt_df = df[df["Year"] == 1962]
sns.scatterplot(data = filt_df, x= "CO2 emissions (metric tons per capita)", y = "gdpPercap")

<AxesSubplot:xlabel='CO2 emissions (metric tons per capita)', ylabel='gdpPercap'>

In [6]:
# remove rows if one or other has nas
nas = filt_df["gdpPercap"].isna() | filt_df["CO2 emissions (metric tons per capita)"].isna()
filt_df = filt_df[~nas]

In [7]:
assert(len(filt_df["CO2 emissions (metric tons per capita)"]) == len(filt_df["gdpPercap"]))

In [8]:
# 3
# https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
covariance = np.cov(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])

# Relationship between CO2 emissions and gdpPercap is positive. Both change in same direction.
covariance

array([[2.46358644e+01, 4.48639240e+04],
       [4.48639240e+04, 9.52638497e+07]])

In [9]:
p_corr_coeff = covariance[1,0] /(np.std(filt_df["CO2 emissions (metric tons per capita)"]) * np.std(filt_df["gdpPercap"]))
p_corr_coeff

0.9347366414038345

In [10]:
r, prob = pearsonr(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
# Strong correlation between CO2 emissions per cap and gross domestic product per cap
# w/low prob of producing r by random chance from uncorr data
r, prob

(0.926081672501947, 1.1286792210040484e-46)

In [11]:
# 4
years = df["Year"].unique()
years

array([1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002, 2007],
      dtype=int64)

In [26]:
# create functions to repeat data processing and stat calculations
def filter_df_year(df, year):
    filt_df = df[df["Year"] == year]
    # remove rows if one or other has nas
    nas = filt_df["gdpPercap"].isna() | filt_df["CO2 emissions (metric tons per capita)"].isna()
    filt_df = filt_df[~nas]
    return filt_df

def yearly_correlation(df, year):
    filt_df = filter_df_year(df, year)
    r, prob = pearsonr(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])
    return r, prob

In [27]:
r_for_year = [yearly_correlation(df, year)[0] for year in years]

In [28]:
highest_r_year = years[list(r_for_year).index(max(r_for_year))]

In [30]:
df_h_r = filter_df_year(df, highest_r_year)

fig = px.scatter(df_h_r, 
                 x = df_h_r["CO2 emissions (metric tons per capita)"], y = df_h_r["gdpPercap"], 
                 size = df_h_r["pop"], color = df_h_r["continent"])
fig.show()